### Connecting to the service

To start working with the flight fusion service, let's start a local instance.

```sh
fusion server start
```

In [ ]:
from flight_fusion import FusionServiceClient, ClientOptions
from flight_fusion.ipc.v1alpha1 import AreaSourceMetadata, SaveMode
import numpy as np
import pandas as pd
import warnings
import dagster

warnings.filterwarnings("ignore", category=dagster.ExperimentalWarning)

# lets also define some test data to interact with the service
np.random.seed(42)
df_example = pd.DataFrame(np.random.randn(5, 3), columns=["col1", "col2", "col3"])

# set connection options
options = ClientOptions(host="localhost", port=50051)

# and create an instance of the service client
ffc = FusionServiceClient(options)

### Writing and reading data

#### Dataset Client

In [ ]:
# when interacting with a single dataset, you need a dataset client
fds = ffc.get_dataset_client(name="new_dataset", areas=["demo"])

# write data into dataset
fds.write_into(df_example, SaveMode.SAVE_MODE_OVERWRITE)

In [ ]:
# to retrieve data, you can load the entire table ...
df = fds.load().to_pandas()

# ... or execute a query against it
df_query = fds.query("SELECT avg(col1) FROM new_dataset").to_pandas()

df.shape, df_query.shape

#### Versioned Dataset Client

In [ ]:
# getting a client for versioned or regular datasets works the same way
vds = ffc.get_versioned_dataset_client(name="new_versioned_dataset", areas=["demo"])

vds.write_into(df_example, SaveMode.SAVE_MODE_OVERWRITE)

In [ ]:
df = vds.load().to_pandas()
df.head()

#### Querying multiple datasets

In [ ]:
fds_1 = ffc.get_dataset_client(name="table_1", areas=[])
fds_1.write_into(df_example)

fds_2 = ffc.get_dataset_client(name="table_2", areas=[])
fds_2.write_into(df_example)

ctx = ffc.get_context([("table_1", []), ("table_2", [])])
query = "SELECT * FROM table_1 UNION ALL SELECT * FROM table_2"
df = ctx.query(query).to_pandas()

df

### Discovery and metadata

In [ ]:
fds.schema()